In [1]:
import pandas as pd
import numpy as np


In [2]:
# # Here we generate randomly historical data in year 2022

# start_date = pd.Timestamp(year=2022, month=1, day=1, hour=0)
# ending_date = pd.Timestamp(year=2022, month=12, day=31, hour=23)

# random_timestamps_int = np.random.randint(0, int((ending_date - start_date).total_seconds()/3600), 1000)
# random_dates = start_date + pd.to_timedelta(random_timestamps_int, unit='H')
# location_point = np.random.randint(1, 30, size=(1000, 2))

# # Then combine them in one dateframe
# random_dates_df = pd.DataFrame(random_dates, columns=['starting_time'])
# location_point_df = pd.DataFrame(location_point, columns=['Origin_point', 'Destination_point'])
# combined_df = pd.concat([random_dates_df, location_point_df], axis=1)




In [3]:
# combined_df.to_excel('dl test example.xlsx', index=False)

In [4]:
data = pd.read_excel('dl test example.xlsx')

In [5]:
data['starting_time'] = pd.to_datetime(data['starting_time'])
data['day_of_week'] = data['starting_time'].dt.dayofweek
data['hour'] = data['starting_time'].dt.hour
data['time_interval'] = data['hour'] // 8



In [6]:
aggregated_data = data.groupby(['Origin_point', 'Destination_point', 'day_of_week', 'time_interval']).size().reset_index(name='demand')

In [7]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

#By doing this, all the features are now on a similar scale, typically between 0 and 1, 
#which can lead to better performance and faster convergence of the model during training.
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(data[['Origin_point', 'Destination_point', 'day_of_week', 'time_interval']]).toarray()

# Scale the date
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(encoded_features)

In [10]:
print(len(aggregated_data['demand']))

978


In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, aggregated_data['demand'], test_size=0.2, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [1000, 978]

In [ ]:
# Reshape input to be [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
# Step 2: Traffic Demand Prediction Model

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')